In [45]:
import os
import re
import ast
import pandas as pd
from tqdm.notebook import tqdm
tqdm.pandas()

In [171]:
def get_api_path(api_full_path_name):
    lib_name = api_full_path_name.split('.')[0].strip()
    lib_path = f'data/{lib_name}/'
    folders = [x for x in os.listdir(lib_path) if 
               not x.endswith('.tar.gz') and not x.endswith('zip') and x.startswith(lib_name)]
    #remove leading 'lib_name' from folder path
    api_path = '/'.join(api_full_path_name.split('.')[:-1])
    api_name = api_full_path_name.split('.')[-1]
#     if len(api_full_path_name.split('.'))==2:
#         return [], api_name
    
    # list all ver_path with api_path
    dir_paths = [x[0] for x in os.walk(os.path.join(lib_path)) if x[0].endswith(api_path)]

    # find py files that defines the api
    api_path = []
    for path in dir_paths:
        grep_result = !grep -r 'def {api_name}' {path}/*
        api_path.extend([x.split(':')[0] for x in grep_result])
    
#     if unable to find definition of function, look for funtion by full name 
    if not api_path:
        grep_result = !grep -r '{api_full_path_name}' {path}/*
        api_path.extend([x.split(':')[0] for x in grep_result])
        
    # try searching without the lib name    
    if not api_path:
        partial_name = '.'.join(api_full_path_name.split('.')[1:])
        grep_result = !grep -r '{partial_name}' {path}/*
        api_path.extend([x.split(':')[0] for x in grep_result])
    
    # global search. file_path do not need to end with the api_path
    if not api_path:
        for path in folders:
            grep_result = !grep -r 'def {api_name}' {path}/*
            api_path.extend([x.split(':')[0] for x in grep_result])
        
    return api_path, api_name
# print(get_api_path("scipy.misc.bytescale"))
# print(get_api_path('scipy.interpolate.splmake'))
# print(get_api_path("scipy.stats.cmedian"))
# print(get_api_path("seaborn.corrplot"))
print(get_api_path('numpy.mirr'))



(['data/numpy/numpy-1.7.0/numpy/lib/financial.py', 'data/numpy/numpy-1.10.0.post2/numpy/lib/financial.py', 'data/numpy/numpy-1.9.1/numpy/lib/financial.py', 'data/numpy/numpy-1.16.0rc1/numpy/lib/financial.py', 'data/numpy/numpy-1.7.1/numpy/lib/financial.py', 'data/numpy/numpy-1.9.0/numpy/lib/financial.py', 'data/numpy/numpy-1.16.3/numpy/lib/financial.py', 'data/numpy/numpy-1.16.4/numpy/lib/financial.py', 'data/numpy/numpy-1.14.0/numpy/lib/financial.py', 'data/numpy/numpy-1.13.0rc2/numpy/lib/financial.py', 'data/numpy/numpy-1.18.5/numpy/lib/financial.py', 'data/numpy/numpy-1.18.2/numpy/lib/financial.py', 'data/numpy/numpy-1.10.1/numpy/lib/financial.py', 'data/numpy/numpy-1.17.0rc1/numpy/lib/financial.py', 'data/numpy/numpy-1.14.6/numpy/lib/financial.py', 'data/numpy/numpy-1.14.1/numpy/lib/financial.py', 'data/numpy/numpy-1.16.5/numpy/lib/financial.py', 'data/numpy/numpy-1.16.2/numpy/lib/financial.py', 'data/numpy/numpy-1.18.3/numpy/lib/financial.py', 'data/numpy/numpy-1.18.4/numpy/lib/fi

In [95]:
# api_full_path_name = 'numpy.ma.loads'
# api_path = '/'.join(api_full_path_name.split('.')[:-1])
# api_name = api_full_path_name.split('.')[-1]
# folders = [x for x in os.listdir(lib_path) if 
#                not x.endswith('.tar.gz') and not x.endswith('zip') and x.startswith(lib_name)]
# api_path = []
# for i in folders:
#     grep_result = !grep -r 'def {api_name}' data/numpy/{i}/numpy/*
#     api_path.extend([x.split(':')[0] for x in grep_result])
    
# for path in api_path:
#     try:
#         code = open(path, 'r').read()
#     except:
#         continue
#     print(path)
#     code = code.replace('\n', '').replace('\r', '')
#     messages = re.findall('@*deprecate*\((.*?)\)', code)
#     if not messages:
#         messages = re.findall('.warn*\((.*?)\)', code)
#     if messages:
#         print(messages)



In [170]:
# look into each of them for deprecation 


def parse_deprecation_declarator(api_path, api_name):
    deprecation_declarator_messages = []
    for path in api_path:
        try:
            code = open(path, 'r').read()
        except:
            continue
        code = code.replace('\n', '').replace('\r', '')
        
        messages = re.findall('@*deprecate*\((.*?)\)', code)
        if not messages:
            messages = re.findall('.warn*\((.*?)\)', code)
        deprecation_declarator_messages.extend(messages)
        
    api_name_hits = [x for x in deprecation_declarator_messages if api_name in x]
    if not api_name_hits:
        return set(deprecation_declarator_messages)
    return set(api_name_hits)

def test_declarator_parse(api_full_path_name):
    api_path, api_name = get_api_path(api_full_path_name)
    return parse_deprecation_declarator(api_path, api_name)

test_declarator_parse('numpy.ma.MaskedArray.mini')

set()

In [162]:
'.a.s.d.'.strip('.')

'a.s.d'

In [167]:
def select_replacements_from_messages(api_name, messages):
    replacements = set()
    api_name = api_name.lower()
    messages = [ ' '.join(x.lower().replace('"','').split()) for x in messages]
    #using double quotes
    for message in messages:
        if api_name in message:
#             for lib in re.findall('`(.*?)`',message):
#                 replacements.add(lib)
            for lib in re.findall('``(.*?)``',message): 
                replacements.add(lib)
    
    #pattern: use <> instead
    if not replacements:
        for message in messages:
            if api_name in message:
                for lib in re.findall('use (.*?) instead',message.replace('`', '')): 
                    replacements.add(lib)
    
    
    #pattern: "use instead <> " 
    if not replacements:
        for message in messages:
            if api_name in message:
                for lib in re.findall('use instead (.*) ',message.replace('`', '')): 
                    replacements.add(lib)
                    
    #pattern: "use <> " 
    if not replacements:
        for message in messages:
            if api_name in message:
                for lib in re.findall('use (.*?) ',message.replace('`', '')): 
                    replacements.add(lib)
                    
    #pattern: use <> <end>
    if not replacements:
        for message in messages:
            if api_name in message:
                for lib in re.findall('use (.*)',message.replace('`', '')): 
                    replacements.add(lib)
                    
    #pattern: in favor of
    if not replacements:
        for message in messages:
            if api_name in message:
                for lib in re.findall('in favor of (.*?) ',message.replace('`', '')): 
                    replacements.add(lib)
                for lib in re.findall('in favor of (.*?).',message.replace('`', '')): 
                    replacements.add(lib)
                    
    #pattern ' <> instead'
    if not replacements:
        for message in messages:
            if api_name in message:
                if ' instead' in message:
                    words = message.replace('`', '').split(' ')
                    for i in range(len(words)):
                        if 'instead' in words[i]:
                            replacements.add(words[i-1])
                            
    #get any dot seperated words names if api_name is mentioned
    if not replacements:
        for message in messages:
            if api_name in message:
                words = message.replace('`', '').split(' ')
                for word in words:
                    if '.' in word.strip('.'):
                        replacements.add(word)
    
    
    return replacements


def test_declarator_select(api_full_path):
    api_name = api_full_path.split('.').pop()
    messages = test_declarator_parse(api_full_path)
    candidates = select_replacements_from_messages(api_name, messages)
    return list(candidates)
    

# test_declarator_select('scipy.interpolate.splmake')
# test_declarator_select('scipy.stats.cmedian')


# messages = {"""'message="`fromimage` is deprecated in SciPy 1.0.0. "                       
#             "and will be removed in 1.2.0.\\n"                         "Use ``np.asarray(im'"""}
# select_replacements_from_messages("fromimage", messages)

# messages = {'message="scipy.constants.K2F is deprecated in scipy 0.18.0. "                       "Use scipy.constants.convert_teperature instead. "                       "Note that the new function has a different signature."', 'message="scipy.constants.K2F is deprecated in scipy 0.18.0. "                       "Use scipy.constants.convert_temperature instead. "                       "Note that the new function has a different signature."'}
# select_replacements_from_messages("K2F", messages)
# test_declarator_select('numpy.ma.loads')

In [159]:
api_name = ''
message = 'importing from numpy.testing.utils is deprecated, import from numpy.testing instead.'
# print(message.replace('`', ''))
# print(re.findall(' (.*?) instead', message.replace('`', '')))
# if api_name in message:
#     print(message)
#     if ' instead' in message:
#         words = message.replace('`', '').split(' ')
#         print(words)
#         print(words[words.index('instead')-1])
print(re.findall('\w+(?=\s+instead)',' import numpy.testing instead.'.replace('`', '')))
# print(re.findall(' (.*?) instead','import from abc instead.'.replace('`', '')))
# print(re.findall(' (.*?) instead',' asd asd from numpy.testing instead.'.replace('`', '')))

['testing']


In [54]:
lib = 'numpy'
df = pd.read_csv(f'./data/{lib}.csv')

In [55]:
df

,deprecated API,replacement API
0,numpy.distutils.exec_command,subprocess.Popen
1,numpy.alen,numpy.len
2,numpy.core.typeNA,numpy.sctypeDict
3,numpy.core.sctypeNA,numpy.sctypeDict
4,numpy.testing.utils,numpy.testing
5,numpy.testing.decorators,numpy.testing
6,numpy.testing.nosetester,numpy.testing
7,numpy.testing.noseclasses,numpy.testing
8,numpy.FloatFormat,numpy.FloatingFormat
9,numpy.LongFloatFormat,numpy.FloatingFormat


In [106]:

df = df[~df['deprecated API'].str.contains(':')]
print('Total number of applicable APIs', len(df))
df_with_replacements = df[~((df['replacement API'].isna()) |
                            (df['replacement API']=='') |
                            (df['replacement API']==' ') |
                            (df['replacement API']=='no') |
                            (df['replacement API'].str.contains(' ')) |
                            (df['replacement API'].str.contains('-')))
                         ]
print('Total number of WITH APIs replacements ', len(df_with_replacements))

df_with_replacements['extracted_text'] = df_with_replacements[
    'deprecated API'].progress_apply(test_declarator_parse)

Total number of applicable APIs 44
Total number of WITH APIs replacements  44


  0%|          | 0/44 [00:00<?, ?it/s]

In [168]:
df_with_replacements['PROPOSED_REPLACEMENT'] = df_with_replacements.progress_apply(
    lambda x: select_replacements_from_messages(x['deprecated API'].split('.').pop(), x.extracted_text)
    , axis=1)
df_with_replacements.to_csv(f'./results/{lib}-results.csv', index=False)

  0%|          | 0/44 [00:00<?, ?it/s]

In [169]:
df_with_replacements

,deprecated API,replacement API,extracted_text,PROPOSED_REPLACEMENT
0,numpy.distutils.exec_command,subprocess.Popen,"{'_exec_command_posix failed (status=%s, 'exec...",{' 'subprocess.popen}
1,numpy.alen,numpy.len,"{ ""`np.alen` is deprecated, use `len` i...",{len}
2,numpy.core.typeNA,numpy.sctypeDict,{},{}
3,numpy.core.sctypeNA,numpy.sctypeDict,{},{}
4,numpy.testing.utils,numpy.testing,"{""Importing from numpy.testing.utils is deprec...",{numpy.testing}
5,numpy.testing.decorators,numpy.testing,"{""Importing from numpy.testing.decorators is d...",{numpy.testing}
6,numpy.testing.nosetester,numpy.testing,"{""Importing from numpy.testing.nosetester is d...",{numpy.testing}
7,numpy.testing.noseclasses,numpy.testing,"{""Importing from numpy.testing.noseclasses is ...",{numpy.testing}
8,numpy.FloatFormat,numpy.FloatingFormat,{},{}
9,numpy.LongFloatFormat,numpy.FloatingFormat,{},{}


In [109]:
# df_with_replacements['paths'] = df_with_replacements['deprecated API'].progress_apply(
#     get_api_path)

In [110]:
df_with_replacements

,deprecated API,replacement API,extracted_text,PROPOSED_REPLACEMENT
0,numpy.distutils.exec_command,subprocess.Popen,"{'_exec_command_posix failed (status=%s, 'exec...",{' 'subprocess.popen}
1,numpy.alen,numpy.len,"{ ""`np.alen` is deprecated, use `len` i...",{len}
2,numpy.core.typeNA,numpy.sctypeDict,{},{}
3,numpy.core.sctypeNA,numpy.sctypeDict,{},{}
4,numpy.testing.utils,numpy.testing,"{""Importing from numpy.testing.utils is deprec...",{}
5,numpy.testing.decorators,numpy.testing,"{""Importing from numpy.testing.decorators is d...",{}
6,numpy.testing.nosetester,numpy.testing,"{""Importing from numpy.testing.nosetester is d...",{}
7,numpy.testing.noseclasses,numpy.testing,"{""Importing from numpy.testing.noseclasses is ...",{}
8,numpy.FloatFormat,numpy.FloatingFormat,{},{}
9,numpy.LongFloatFormat,numpy.FloatingFormat,{},{}


In [62]:
import time
for i, j in zip(df_with_replacements['deprecated API'], df_with_replacements['extracted_text']):
    print(i, j)
    time.sleep(1)

numpy.distutils.exec_command {"'_exec_command_posix failed (status=%s", "'exec_command is deprecated since NumPy v1.17, use '                  'subprocess.Popen instead', DeprecationWarning, stacklevel=1"}
numpy.alen {'        "`np.alen` is deprecated, use `len` instead",        DeprecationWarning, stacklevel=2'}
numpy.core.typeNA set()
numpy.core.sctypeNA set()
numpy.testing.utils {'"Importing from numpy.testing.utils is deprecated, "              "import from numpy.testing instead.",              ImportWarning, stacklevel=2'}
numpy.testing.decorators {'"Importing from numpy.testing.decorators is deprecated, "              "import from numpy.testing instead.",              DeprecationWarning, stacklevel=2'}
numpy.testing.nosetester {'"Importing from numpy.testing.nosetester is deprecated, "              "import from numpy.testing instead.",              DeprecationWarning, stacklevel=2'}
numpy.testing.noseclasses {'"Importing from numpy.testing.noseclasses is deprecated, "            

numpy.ma.MaskedArray.mini set()
numpy.lib.function_base.unique set()
numpy.rank {'        "`rank` is deprecated; use the `ndim` attribute or function instead. "        "To find the rank of a matrix see `numpy.linalg.matrix_rank`.",        VisibleDeprecationWarning, stacklevel=2', '        "`rank` is deprecated; use the `ndim` attribute or function instead. "        "To find the rank of a matrix see `numpy.linalg.matrix_rank`.",        VisibleDeprecationWarning, stacklevel=3', '        "`rank` is deprecated; use the `ndim` function instead. ",        np.VisibleDeprecationWarning', '        "`rank` is deprecated; use the `ndim` attribute or function instead. "        "To find the rank of a matrix see `numpy.linalg.matrix_rank`.",        VisibleDeprecationWarning', '        "`rank` is deprecated; use the `ndim` function instead. ",        np.VisibleDeprecationWarning, stacklevel=2'}
numpy.testing.rand {'message="numpy.testing.rand is deprecated in numpy 1.11. "                   "Use nump

In [15]:
import seaborn
seaborn.corrplot

AttributeError: module 'seaborn' has no attribute 'corrplot'

In [ ]:
# df_with_replacements[df_with_replacements['PROPOSED_REPLACEMENT'].apply(lambda x: x!=[])]

In [ ]:
df_with_replacements

In [ ]:
# DeprecationWarning #this does not work
# def parse_deprecation_warning(api_path):
#     deprecation_warning_messages = []
#     for path in api_path:
#         code = open(path, 'r').read()
#         code = code.replace('\n', '').replace('\r', '')
#         if 'warnings.warn' in code:
#             print(code)
#         messages = re.findall('warnings.warn*\((.*?)\)', code)
#         deprecation_warning_messages.extend(messages)
#     return set(deprecation_warning_messages)
# def test_depwarn_parse():
#     api_full_path_name = 'scipy.stats.itemfreq'
#     api_path, api_name = get_api_path(api_full_path_name)
#     print(api_path)
#     return parse_deprecation_warning(api_path)
# test_depwarn_parse()

In [ ]:

# parse_deprecation_declarator(api_path)            
parse_deprecation_declarator(get_api_path('scipy.misc.imread'))
select_replacements_from_messages()